<a href="https://colab.research.google.com/github/nilmeg0105/Trade-Resilience-and-Economic-Networks-Modelling/blob/main/DPL_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

fp='Core_economic_indicators (1).csv'
d=pd.read_csv(fp)

# drop junk rows
d=d.dropna(subset=['Country Name'])
d=d[~d['Country Name'].str.contains('Data from database|Last Updated',na=False)]

yrs=[c for c in d.columns if '[YR' in c]
ymap={c:c.split()[0] for c in yrs}
d.rename(columns=ymap,inplace=True)

idv=['Country Name','Country Code','Series Name','Series Code']
valv=[str(y) for y in range(2000,2025)]
dl=pd.melt(d,id_vars=idv,value_vars=valv,var_name='Year',value_name='val')

dl['val']=dl['val'].replace('..',np.nan)  # replace weird ..
dl['val']=pd.to_numeric(dl['val'],errors='coerce')

# pivot to wide
dp=dl.pivot_table(index=['Country Name','Country Code','Year'],columns='Series Code',values='val').reset_index()

cr={'NE.IMP.GNFS.ZS':'imports_goods_services_gdp_pct','NE.EXP.GNFS.ZS':'exports_goods_services_gdp_pct',
'NE.TRD.GNFS.ZS':'trade_gdp_pct','FP.CPI.TOTL.ZG':'inflation_consumer_prices_pct','NY.GDP.MKTP.KD.ZG':'gdp_growth_pct',
'NY.GDP.PCAP.CD':'gdp_per_capita_current_usd','NY.GDP.MKTP.CD':'gdp_current_usd'}
dp.rename(columns=cr,inplace=True)

dp['Year']=dp['Year'].astype(int)
numc=list(cr.values())
dp[numc]=dp[numc].astype(float)

# focus only on some countries
cc=['India','USA','Russia','France','Germany','Italy','China','Japan','Argentina','Portugal','Spain','Croatia','Belgium',
'Australia','Pakistan','Afghanistan','Israel','Iran','Iraq','Bangladesh','Sri Lanka','Canada','UK','Sweden','Saudi Arabia']
dp=dp[dp['Country Name'].isin(cc)]

iso={'USA':'USA','RUS':'RUS','FRA':'FRA','DEU':'DEU','ITA':'ITA','CHN':'CHN','JPN':'JPN','ARG':'ARG','PRT':'PRT','ESP':'ESP',
'HRV':'HRV','BEL':'BEL','AUS':'AUS','PAK':'PAK','AFG':'AFG','ISR':'ISR','IRN':'IRN','IRQ':'IRQ','BGD':'BGD','LKA':'LKA','CAN':'CAN','GBR':'GBR','SWE':'SWE','SAU':'SAU'}
dp['ISO']=dp['Country Code'].map(iso).fillna(dp['Country Code'])

dp.drop_duplicates(subset=['Country Name','Year'],inplace=True)
dp['is_outlier']=((dp['gdp_growth_pct']>20)|(dp['gdp_growth_pct']<-20)).astype(int) # flag crazy values

dp.sort_values(['Country Name','Year'],inplace=True)
dp[numc]=dp.groupby('Country Name')[numc].transform(lambda x:x.interpolate(method='linear'))
dp[numc]=dp.groupby('Country Name')[numc].transform(lambda x:x.fillna(x.mean()))  # still missing? use mean

sl=['Country Name','Country Code','ISO','Year','imports_goods_services_gdp_pct','exports_goods_services_gdp_pct',
'trade_gdp_pct','inflation_consumer_prices_pct','gdp_growth_pct','gdp_per_capita_current_usd','gdp_current_usd']
dc=dp[sl].copy()

dc['trade_balance_gdp_pct']=dc['exports_goods_services_gdp_pct']-dc['imports_goods_services_gdp_pct']
dc['gdp_growth_stability']=dc.groupby('Country Name')['gdp_growth_pct'].transform(lambda x:x.rolling(window=3,min_periods=1).std())
dc['inflation_stability']=dc.groupby('Country Name')['inflation_consumer_prices_pct'].transform(lambda x:x.rolling(window=3,min_periods=1).std())
dc['log_gdp_current_usd']=np.log1p(dc['gdp_current_usd'])
dc['log_gdp_per_capita']=np.log1p(dc['gdp_per_capita_current_usd'])

# temp placeholders
dc['trade_vulnerability_index']=np.nan
dc['economic_shock_sensitivity']=0.4*dc['gdp_growth_stability']+0.4*dc['inflation_stability']+0.2*dc['trade_balance_gdp_pct']
dc['per_capita_trade_intensity']=np.nan

dc['data_source']='core_economic_indicators'

print("shp:",dc.shape)
print("nulls:\n",dc.isnull().sum()) # quick check
print("desc:\n",dc.describe())

op='cleaned_core_economic_indicators.csv'
dc.to_csv(op,index=False)
print("saved:",op)


Cleaned DataFrame Shape: (525, 20)
Missing Values:
 Series Code
Country Name                        0
Country Code                        0
ISO                                 0
Year                                0
imports_goods_services_gdp_pct      0
exports_goods_services_gdp_pct      0
trade_gdp_pct                       0
inflation_consumer_prices_pct      25
gdp_growth_pct                      0
gdp_per_capita_current_usd          0
gdp_current_usd                     0
trade_balance_gdp_pct               0
gdp_growth_stability               21
inflation_stability                45
log_gdp_current_usd                 0
log_gdp_per_capita                  0
trade_vulnerability_index         525
economic_shock_sensitivity         45
per_capita_trade_intensity        525
data_source                         0
dtype: int64
Summary Stats:
 Series Code        Year  imports_goods_services_gdp_pct  \
count         525.00000                      525.000000   
mean         2012.00000      

In [ ]:
import pandas as pd
import numpy as np

path="crop_and_livestock (1).csv"
df=pd.read_csv(path)
df.columns=df.columns.str.strip()

cols=["Domain Code","Domain","Area Code (M49)","Area","Element Code","Element","Item Code (CPC)","Item",
"Year Code","Year","Unit","Value","Flag","Flag Description","Note"]
df=df[[c for c in cols if c in df.columns]].copy()

df["Year"]=pd.to_numeric(df["Year"],errors="coerce").astype("Int64")
df["Value"]=pd.to_numeric(df["Value"],errors="coerce")

focus=["India","USA","Russia","France","Germany","Italy","China","Japan",
"Argentina","Portugal","Spain","Croatia","Belgium","Australia","Pakistan",
"Afghanistan","Israel","Iran","Iraq","Bangladesh","Sri Lanka","Canada",
"UK","Sweden","Saudi Arabia"]
df=df[df["Area"].isin(focus)]

# area harvested (make sure in ha)
a=df[df["Element"]=="Area harvested"].copy()
a["area_ha"]=a["Value"]
a.loc[a["Unit"].str.contains("1000",case=False,na=False),"area_ha"]=a["Value"]*1000
areas=a[["Area","Item","Year","area_ha"]]

# yields to kg/ha
y=df[df["Element"]=="Yield"].copy()
y["yield_kg_ha"]=np.nan
u=y["Unit"].str.lower()
y.loc[u.str.contains("kg/ha",na=False),"yield_kg_ha"]=y["Value"]
y.loc[u.str.contains("hg/ha",na=False),"yield_kg_ha"]=y["Value"]*0.1
y.loc[u.str.contains("t/ha",na=False)|u.str.contains("tonnes per hectare",na=False),"yield_kg_ha"]=y["Value"]*1000
yields=y[["Area","Item","Year","yield_kg_ha"]]

# production numbers directly
p=df[df["Element"]=="Production"].copy()
p2=p[["Area","Item","Year","Value"]].rename(columns={"Value":"reported_production_tonnes"})

# estimate prod by yield*area
base=pd.merge(yields,areas,on=["Area","Item","Year"],how="outer")
base["yield_kg_ha"]=base.groupby(["Area","Item"])["yield_kg_ha"].transform(lambda x:x.interpolate(method="linear"))
base["area_ha"]=base.groupby(["Area","Item"])["area_ha"].transform(lambda x:x.interpolate(method="linear"))
base["estimated_production_tonnes"]=(base["yield_kg_ha"]*base["area_ha"])/1000
prod=base[["Area","Item","Year","estimated_production_tonnes","yield_kg_ha","area_ha"]]

if not prod.empty:
    prod["_w"]=prod["area_ha"].fillna(0)
    prod["_y_w"]=prod["yield_kg_ha"].fillna(0)*prod["_w"]
    w=prod.groupby(["Area","Year"],as_index=False).agg(
        total_area_harvested_ha=("area_ha","sum"),
        est_ag_production_tonnes=("estimated_production_tonnes","sum"),
        _y_w_sum=("_y_w","sum"),
        _w_sum=("_w","sum")
    )
    w["avg_crop_yield_kg_ha"]=np.where(w["_w_sum"]>0,w["_y_w_sum"]/w["_w_sum"],np.nan)
    w=w.drop(columns=["_y_w_sum","_w_sum"])
else:
    w=pd.DataFrame(columns=["Area","Year","total_area_harvested_ha","est_ag_production_tonnes","avg_crop_yield_kg_ha"])

agg=pd.merge(w,p2.groupby(["Area","Year"],as_index=False).agg(
    reported_production_tonnes=("reported_production_tonnes","sum")
),on=["Area","Year"],how="outer")

agg=agg.rename(columns={"Area":"Country"})
agg=agg.sort_values(["Country","Year"]).reset_index(drop=True)

for c in ["total_area_harvested_ha","est_ag_production_tonnes","reported_production_tonnes"]:
    if c in agg.columns: agg[c]=agg[c].fillna(0)

# extra indicators
yagg=yields.groupby(["Area","Year"]).agg(yield_kg_ha=("yield_kg_ha","mean")).reset_index()
yagg=yagg.rename(columns={"Area":"Country"})
agg=pd.merge(agg,yagg,on=["Country","Year"],how="left")

agg["is_yield_outlier"]=(agg["yield_kg_ha"]>10000).astype(int)
agg["production_discrepancy_flag"]=np.where(
    (agg["est_ag_production_tonnes"]>0)&(agg["reported_production_tonnes"]>0)&
    (abs(agg["est_ag_production_tonnes"]-agg["reported_production_tonnes"])/agg["reported_production_tonnes"]>0.1),1,0
)

agg["food_production_index"]=np.where(
    agg["total_area_harvested_ha"]>0,
    agg["avg_crop_yield_kg_ha"]*agg["total_area_harvested_ha"]/1000,
    np.nan
)

agg["yield_variability"]=agg.groupby("Country")["yield_kg_ha"].transform(lambda x:x.rolling(window=3,min_periods=1).std())

# crop diversity
div=prod.groupby(["Area","Year"]).agg(crop_diversity_index=("Item","nunique")).reset_index()
div=div.rename(columns={"Area":"Country"})
agg=pd.merge(agg,div,on=["Country","Year"],how="left")

agg["agricultural_export_potential"]=np.nan

out="crop_livestock_processed.csv"
agg.to_csv(out,index=False)
print("Saved:",out)
print(agg.head(10))


Saved: crop_livestock_processed.csv
       Country  Year  total_area_harvested_ha  est_ag_production_tonnes  \
0  Afghanistan  2000                2800315.0              4.355680e+06   
1  Afghanistan  2001                2484514.0              4.547215e+06   
2  Afghanistan  2002                2627078.0              6.179899e+06   
3  Afghanistan  2003                3390751.0              6.981472e+06   
4  Afghanistan  2004                3036039.0              5.954611e+06   
5  Afghanistan  2005                3471747.0              8.125423e+06   
6  Afghanistan  2006                3418127.0              7.447336e+06   
7  Afghanistan  2007                3463717.0              8.666459e+06   
8  Afghanistan  2008                3152361.0              6.751566e+06   
9  Afghanistan  2009                3649144.0              9.641292e+06   

   avg_crop_yield_kg_ha  reported_production_tonnes  yield_kg_ha  \
0           1555.424956                  7406816.82  5751.545714   
1 

In [ ]:
import pandas as pd, numpy as np

d=pd.read_csv("Resiliance.csv")   # load raw data

d=d.replace("..",np.nan)  # handle weird missing vals

# reshape wide yrs into long tidy format
dl=d.melt(id_vars=["Country Name","Country Code","Series Name","Series Code"],
var_name="Year",value_name="Val")

dl["Year"]=dl["Year"].str.extract(r"(\d{4})").astype(int)  # clean year col
dl["Val"]=pd.to_numeric(dl["Val"],errors="coerce")  # convert vals

dl=dl.sort_values(["Country Name","Series Name","Year"])  # keep in order
dl["miss"]=dl["Val"].isna()  # mark which ones were missing

# fill: interpolate tiny gaps, then forward/back fill edges
def fillf(s): return s.interpolate("linear",limit=1).ffill().bfill()

dl["Val"]=dl.groupby(["Country Name","Series Name"])["Val"].transform(fillf)
dl["miss"]=dl["miss"] & dl["Val"].notna()  # update imputed flag

dl=dl.reset_index(drop=True)
dl.to_csv("processed_timeseries_clean.csv",index=False)  # save cleaned data

print(dl.head(20))


   Country Name Country Code                         Series Name  \
0   Afghanistan          AFG  Current account balance (% of GDP)   
1   Afghanistan          AFG  Current account balance (% of GDP)   
2   Afghanistan          AFG  Current account balance (% of GDP)   
3   Afghanistan          AFG  Current account balance (% of GDP)   
4   Afghanistan          AFG  Current account balance (% of GDP)   
5   Afghanistan          AFG  Current account balance (% of GDP)   
6   Afghanistan          AFG  Current account balance (% of GDP)   
7   Afghanistan          AFG  Current account balance (% of GDP)   
8   Afghanistan          AFG  Current account balance (% of GDP)   
9   Afghanistan          AFG  Current account balance (% of GDP)   
10  Afghanistan          AFG  Current account balance (% of GDP)   
11  Afghanistan          AFG  Current account balance (% of GDP)   
12  Afghanistan          AFG  Current account balance (% of GDP)   
13  Afghanistan          AFG  Current account ba

In [ ]:
import pandas as pd, numpy as np

d=pd.read_csv("Employment_Unemployment.csv")  # load data

d=d.replace("..",np.nan)  # fix missing vals

# reshape wide yrs to long
dl=d.melt(id_vars=["Country Name","Country Code","Series Name","Series Code"],
var_name="Year",value_name="Val")

dl["Year"]=dl["Year"].str.extract(r"(\d{4})").astype(int)  # keep only year num
dl["Val"]=pd.to_numeric(dl["Val"],errors="coerce")  # force numeric

dl=dl.sort_values(["Country Name","Series Name","Year"])  # order data
dl["miss"]=dl["Val"].isna()  # mark na

# fill small gaps only (linear, max 1 step)
def fillf(s): return s.interpolate("linear",limit=1)

dl["Val"]=dl.groupby(["Country Name","Series Name"])["Val"].transform(fillf)
dl["miss"]=dl["miss"] & dl["Val"].notna()  # update imputed flag

dl=dl.reset_index(drop=True)
dl.to_csv("processed_employment_unemployment.csv",index=False)  # save

print(dl.head(20))


   Country Name Country Code  \
0   Afghanistan          AFG   
1   Afghanistan          AFG   
2   Afghanistan          AFG   
3   Afghanistan          AFG   
4   Afghanistan          AFG   
5   Afghanistan          AFG   
6   Afghanistan          AFG   
7   Afghanistan          AFG   
8   Afghanistan          AFG   
9   Afghanistan          AFG   
10  Afghanistan          AFG   
11  Afghanistan          AFG   
12  Afghanistan          AFG   
13  Afghanistan          AFG   
14  Afghanistan          AFG   
15  Afghanistan          AFG   
16  Afghanistan          AFG   
17  Afghanistan          AFG   
18  Afghanistan          AFG   
19  Afghanistan          AFG   

                                          Series Name           Series Code  \
0   Employment to population ratio, 15+, female (%...  SL.EMP.TOTL.SP.FE.ZS   
1   Employment to population ratio, 15+, female (%...  SL.EMP.TOTL.SP.FE.ZS   
2   Employment to population ratio, 15+, female (%...  SL.EMP.TOTL.SP.FE.ZS   
3   Employm

In [ ]:
import pandas as pd
import numpy as np

def process_social_welfare(input_csv: str, output_csv: str) -> pd.DataFrame:
    # --- Load & normalize ---
    df = pd.read_csv(input_csv)
    df.columns = df.columns.str.strip()
    df = df.replace("..", np.nan)

    id_vars = [c for c in ["Country Name", "Country Code", "Series Name", "Series Code"] if c in df.columns]
    value_vars = [c for c in df.columns if c not in id_vars]

    # --- Wide -> Long (Unpivot) ---
    long = df.melt(id_vars=id_vars, value_vars=value_vars,
                   var_name="Year", value_name="Value")

    long["Year"]  = long["Year"].astype(str).str.extract(r"(\d{4})").astype(int)
    long["Value"] = pd.to_numeric(long["Value"], errors="coerce")

    long = long.sort_values(["Country Name","Country Code","Series Code","Year"]).reset_index(drop=True)
    long["Imputed"] = False
    long["Impute_Method"] = "original"

    # --- Indicator-aware rules ---
    def rules_for(series_code: str):
        code = str(series_code)
        if code.startswith("SP.DYN.LE00"):          # Life expectancy (very smooth)
            return dict(limit=3, fill_edges=True)
        if code == "SP.URB.TOTL.IN.ZS":            # Urban population % (smooth)
            return dict(limit=2, fill_edges=True)
        if code == "SP.POP.GROW":                  # Pop growth % (rate)
            return dict(limit=1, fill_edges=False)
        if code == "SL.UEM.TOTL.ZS":               # Unemployment % (gradual)
            return dict(limit=1, fill_edges=False)
        if code == "SI.POV.GINI":                  # Gini (survey, irregular)
            return dict(limit=0, fill_edges=False)
        if code == "SI.POV.DDAY":                  # Poverty headcount (survey)
            return dict(limit=0, fill_edges=False)
        # Default conservative rule
        return dict(limit=1, fill_edges=False)

    # --- Apply per (country, series) group ---
    out = []
    grp_cols = ["Country Name","Country Code","Series Code","Series Name"]
    for keys, g in long.groupby(grp_cols, sort=False):
        rule = rules_for(g["Series Code"].iloc[0])
        s = g["Value"].copy()
        method = pd.Series("original", index=g.index)

        # Interpolate only up to `limit` consecutive missing years (internal gaps)
        if rule["limit"] > 0:
            s_interp = s.interpolate(method="linear", limit=rule["limit"])
            method.loc[s.isna() & s_interp.notna()] = "interp"
        else:
            s_interp = s

        # Optionally fill edges (start/end) with last/next observation
        if rule["fill_edges"]:
            s_ff = s_interp.ffill()
            method.loc[s_interp.isna() & s_ff.notna()] = "ffill"
            s_fb = s_ff.bfill()
            method.loc[s_ff.isna() & s_fb.notna()] = "bfill"
            s_final = s_fb
        else:
            s_final = s_interp

        g = g.copy()
        g["Value"] = s_final
        g["Imputed"] = method.ne("original")
        g["Impute_Method"] = method
        out.append(g)

    final = pd.concat(out, axis=0).sort_values(
        ["Country Name","Country Code","Series Code","Year"]
    ).reset_index(drop=True)

    final.to_csv(output_csv, index=False)
    return final

if __name__ == "__main__":
    # Change the paths as needed
    processed = process_social_welfare(
        "Social_and_welfare.csv",
        "processed_social_and_welfare.csv"
    )
    print(processed.head(20))


   Country Name Country Code  \
0   Afghanistan          AFG   
1   Afghanistan          AFG   
2   Afghanistan          AFG   
3   Afghanistan          AFG   
4   Afghanistan          AFG   
5   Afghanistan          AFG   
6   Afghanistan          AFG   
7   Afghanistan          AFG   
8   Afghanistan          AFG   
9   Afghanistan          AFG   
10  Afghanistan          AFG   
11  Afghanistan          AFG   
12  Afghanistan          AFG   
13  Afghanistan          AFG   
14  Afghanistan          AFG   
15  Afghanistan          AFG   
16  Afghanistan          AFG   
17  Afghanistan          AFG   
18  Afghanistan          AFG   
19  Afghanistan          AFG   

                                          Series Name  Series Code  Year  \
0   Poverty headcount ratio at $3.00 a day (2021 P...  SI.POV.DDAY  2000   
1   Poverty headcount ratio at $3.00 a day (2021 P...  SI.POV.DDAY  2001   
2   Poverty headcount ratio at $3.00 a day (2021 P...  SI.POV.DDAY  2002   
3   Poverty headcount r

In [ ]:
import pandas as pd

# just keep 25 countries we care abt
cts=["India","USA","Russia","France","Germany","Italy","China","Japan","Argentina","Portugal",
     "Spain","Croatia","Belgium","Australia","Pakistan","Afghanistan","Israel","Iran","Iraq",
     "Bangladesh","Sri Lanka","Canada","UK","Sweden","Saudi Arabia"]

df=pd.read_csv("export_processed.csv",encoding="latin1")
df=df[df["reporterDesc"].isin(cts)]   # filter only chosen

df.to_csv("export_processed.csv",index=False)
print("export_processed.csv updated with 25 countries only")

# find per-year max TDI (kinda like worst case dep)
mx=df.groupby(["reporterDesc","refYear"])["TradeDependencyIndex"].max().reset_index()

# avg of those yearly max per country
avg=mx.groupby("reporterDesc")["TradeDependencyIndex"].mean().reset_index()
top3=avg.sort_values("TradeDependencyIndex",ascending=False).head(3)

print("\nTop 3 Vulnerable Nations by Avg Max TDI:")
print(top3[["reporterDesc","TradeDependencyIndex"]])


export_processed.csv has been updated with only the specified 25 countries.

Top 3 Vulnerable Nations by Average Max Trade Dependency Index:
   reporterDesc  TradeDependencyIndex
5        Canada              0.861269
12         Iraq              0.757560
0   Afghanistan              0.497160


In [ ]:
!pip install Prophet

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 15.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd, numpy as np, tensorflow as tf, optuna
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler

# load data
exp=pd.read_csv("export_processed.csv",encoding="latin1");integ=pd.read_csv("integrated_master.csv",encoding="latin1")
ctrys=["India","USA","Russia","France","Germany","Italy","China","Japan","Argentina","Portugal","Spain","Croatia","Belgium","Australia","Pakistan","Afghanistan","Israel","Iran","Iraq","Bangladesh","Sri Lanka","Canada","UK","Sweden","Saudi Arabia"]
exp=exp[exp["reporterDesc"].isin(ctrys)]

# find vuln
mx=exp.groupby(["reporterDesc","refYear"])["TradeDependencyIndex"].max().reset_index()
av=mx.groupby("reporterDesc")["TradeDependencyIndex"].mean().reset_index()
t3=av.sort_values(by="TradeDependencyIndex",ascending=False).head(3)
print("\nTop 3 vulnerable (by avg yearly max TDI):");print(t3)

# find top partners per country
tp=[]
for c in t3["reporterDesc"]:
    s=exp[exp["reporterDesc"]==c];yr=s["refYear"].max();sl=s[s["refYear"]==yr]
    p=sl.sort_values("TradeDependencyIndex",ascending=False).iloc[0]
    tp.append({"country":c,"partner":p["partnerDesc"],"partner_export_share":p["TradeDependencyIndex"],"year_basis":yr})
tpd=pd.DataFrame(tp)
print("\nTop partner & export share (latest year):");print(tpd)

# helper for sequence making
def mkseq(X,y,w=3):
    xx,yy=[],[]
    for i in range(len(X)-w): xx.append(X[i:i+w]);yy.append(y[i+w])
    return np.array(xx),np.array(yy)

# optuna obj (bad naming, minimal doc)
def obj(tr,Xtr,ytr,Xv,yv,inp):
    u=tr.suggest_int("units",32,128);l=tr.suggest_int("layers",1,3);d=tr.suggest_float("dropout",0.1,0.5);lr=tr.suggest_loguniform("lr",1e-4,1e-2)
    m=keras.Sequential()
    for i in range(l):
        rs=(i<l-1)
        m.add(layers.LSTM(u,activation="tanh",return_sequences=rs,input_shape=inp if i==0 else None))
        m.add(layers.Dropout(d))
    m.add(layers.Dense(1))
    m.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),loss="mse")
    es=keras.callbacks.EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)
    h=m.fit(Xtr,ytr,validation_data=(Xv,yv),epochs=100,batch_size=tr.suggest_categorical("batch_size",[8,16,32]),verbose=0,callbacks=[es])
    return min(h.history["val_loss"])

# run model per country
smry=[]
feats=["imports_goods_services_gdp_pct","exports_goods_services_gdp_pct","trade_gdp_pct","inflation_consumer_prices_pct","gdp_growth_pct","trade_balance_gdp_pct","gdp_growth_stability","inflation_stability","HHI_export","Diversification_export","HHI_import","Diversification_import","Trade_Diversification_Index","Overall_Trade_Dependency"]
tgt="gdp_current_usd"
for _,r in tpd.iterrows():
    c=r["country"];p=r["partner"];shr=r["partner_export_share"]
    print(f"\n==============================");print(f"Country: {c} | Top partner: {p} ({shr:.2f})");print("==============================")
    df=integ[integ["Country Name"]==c].copy();df=df.dropna(subset=feats+[tgt])
    if df.empty or df["Year"].nunique()<10: print(f"Not enough data for {c}. Skipping.");continue
    sx=MinMaxScaler();sy=MinMaxScaler();X=sx.fit_transform(df[feats]);y=sy.fit_transform(df[[tgt]])
    Xs,ys=mkseq(X,y,3)

    if len(Xs)<10:
        print(f"Too short sequence for {c}. Skipping.");
        continue
    yrs=df["Year"].values[3:]
    trcut=np.where(yrs<=2022)[0][-1]
    vcut=np.where(yrs<=2024)[0][-1]
    Xtr,Ytr=Xs[:trcut+1],ys[:trcut+1]
    Xv,Yv=Xs[trcut+1:vcut+1],ys[trcut+1:vcut+1]
    Xt,Yt=Xs[vcut+1:],ys[vcut+1:]

    st=optuna.create_study(direction="minimize");st.optimize(lambda tr: obj(tr,Xtr,Ytr,Xv,Yv,(Xtr.shape[1],Xtr.shape[2])),n_trials=20,timeout=300)
    bp=st.best_params
    print("Best params:",bp)
    bm=keras.Sequential()

    for i in range(bp["layers"]):
        rs=(i<bp["layers"]-1)
        bm.add(layers.LSTM(bp["units"],activation="tanh",return_sequences=rs,input_shape=(Xtr.shape[1],Xtr.shape[2]) if i==0 else None))
        bm.add(layers.Dropout(bp["dropout"]))
    bm.add(layers.Dense(1));bm.compile(optimizer=keras.optimizers.Adam(learning_rate=bp["lr"]),loss="mse")
    es=keras.callbacks.EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)
    bm.fit(np.concatenate([Xtr,Xv]),np.concatenate([Ytr,Yv]),epochs=200,batch_size=bp["batch_size"],verbose=0,callbacks=[es])
    ls=Xs[-1:];bl=bm.predict(ls,verbose=0)[0][0];blv=sy.inverse_transform([[bl]])[0,0]
    sh=ls.copy();idx=feats.index("Overall_Trade_Dependency");sh[0,-1,idx]*=(1-0.40*shr)
    shv=bm.predict(sh,verbose=0)[0][0];shv=sy.inverse_transform([[shv]])[0,0]
    loss=blv-shv;lp=loss/blv*100
    smry.append({"country":c,"top_partner":p,"baseline_gdp_2026":blv,"shock_gdp_2026":shv,"gdp_loss":loss,"gdp_loss_pct":lp})

# print results
if smry:
    sm=pd.DataFrame(smry).sort_values("gdp_loss_pct",ascending=False)
    pd.set_option("display.float_format",lambda v:f"{v:,.2f}")
    print("\n===== GDP Impact Summary (2026, 40% drop in top partner's imports, Tuned LSTM) =====");print(sm)
else: print("\n No valid countries had enough data for tuned LSTM modeling.")


[I 2025-08-19 12:18:36,989] A new study created in memory with name: no-name-feb3b7a7-aa97-46f4-b064-a854d4afd7ec



Top 3 vulnerable (by avg yearly max TDI):
   reporterDesc  TradeDependencyIndex
5        Canada                  0.86
12         Iraq                  0.76
0   Afghanistan                  0.50

Top partner & export share (latest year):
       country partner  partner_export_share  year_basis
0       Canada     USA                  0.85        2024
1         Iraq   Italy                  0.88        2016
2  Afghanistan   India                  0.51        2019

Country: Canada | Top partner: USA (0.85)


[I 2025-08-19 12:18:56,207] Trial 0 finished with value: 0.006466730032116175 and parameters: {'units': 81, 'layers': 3, 'dropout': 0.24039347666068003, 'lr': 0.0016048261989244424, 'batch_size': 8}. Best is trial 0 with value: 0.006466730032116175.
[I 2025-08-19 12:19:12,225] Trial 1 finished with value: 0.01800517365336418 and parameters: {'units': 113, 'layers': 3, 'dropout': 0.4029050399022176, 'lr': 0.0001998041694267846, 'batch_size': 8}. Best is trial 0 with value: 0.006466730032116175.
[I 2025-08-19 12:19:19,190] Trial 2 finished with value: 0.0003266715502832085 and parameters: {'units': 99, 'layers': 2, 'dropout': 0.32678593908266795, 'lr': 0.0011997020628456317, 'batch_size': 32}. Best is trial 2 with value: 0.0003266715502832085.
[I 2025-08-19 12:19:27,964] Trial 3 finished with value: 0.018452726304531097 and parameters: {'units': 66, 'layers': 2, 'dropout': 0.3932050904633957, 'lr': 0.00036921908727854556, 'batch_size': 16}. Best is trial 2 with value: 0.00032667155028320

Best params: {'units': 103, 'layers': 1, 'dropout': 0.3648153263573667, 'lr': 0.0027531239390954114, 'batch_size': 32}

Country: Iraq | Top partner: Italy (0.88)
Not enough data for Iraq. Skipping.

Country: Afghanistan | Top partner: India (0.51)
Too short sequence for Afghanistan. Skipping.

===== GDP Impact Summary (2026, 40% drop in top partner's imports, Tuned LSTM) =====
  country top_partner    baseline_gdp_2026       shock_gdp_2026  \
0  Canada         USA 2,179,241,254,901.17 2,184,160,342,480.90   

           gdp_loss  gdp_loss_pct  
0 -4,919,087,579.73         -0.23  


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import optuna
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler

# load data & filter
e = pd.read_csv("export_processed.csv", encoding="latin1")
i = pd.read_csv("integrated_master.csv", encoding="latin1")
cs = ["India","USA","Russia","France","Germany","Italy","China","Japan","Argentina","Portugal","Spain","Croatia","Belgium","Australia","Pakistan","Afghanistan","Israel","Iran","Iraq","Bangladesh","Sri Lanka","Canada","UK","Sweden","Saudi Arabia"]
e = e[e["reporterDesc"].isin(cs)]

# vuln calc
mx = e.groupby(["reporterDesc","refYear"])["TradeDependencyIndex"].max().reset_index()
av = mx.groupby("reporterDesc")["TradeDependencyIndex"].mean().reset_index()
t3 = av.sort_values(by="TradeDependencyIndex", ascending=False).head(3)
print("\nTop 3 vulnerable (by avg yearly max TDI):")
print(t3)

# partner pick
tp = []
for c in t3["reporterDesc"]:
    s = e[e["reporterDesc"]==c]
    yr = s["refYear"].max()
    sl = s[s["refYear"]==yr]
    p = sl.sort_values("TradeDependencyIndex", ascending=False).iloc[0]
    tp.append({"c":c,"p":p["partnerDesc"],"shr":p["TradeDependencyIndex"],"y":yr})

tpd = pd.DataFrame(tp)
print("\nTop partner & export share (latest year):")
print(tpd)

# seq maker
def ms(X,y,w=3):
    xx,yy = [],[]
    for k in range(len(X)-w):
        xx.append(X[k:k+w])
        yy.append(y[k+w])
    return np.array(xx),np.array(yy)

# optuna obj
def o(tr,Xtr,ytr,Xv,yv,inp):
    u = tr.suggest_int("u",32,128)
    l = tr.suggest_int("l",1,3)
    d = tr.suggest_float("d",0.1,0.5)
    lr = tr.suggest_loguniform("lr",1e-4,1e-2)
    m = keras.Sequential()
    for j in range(l):
        rs = (j<l-1)
        if j==0:
            m.add(layers.LSTM(u,activation="tanh",return_sequences=rs,input_shape=inp))
        else:
            m.add(layers.LSTM(u,activation="tanh",return_sequences=rs))
        m.add(layers.Dropout(d))
    m.add(layers.Dense(1))
    m.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),loss="mse")
    es = keras.callbacks.EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)
    h = m.fit(Xtr,ytr,validation_data=(Xv,yv),epochs=100,batch_size=tr.suggest_categorical("bs",[8,16,32]),verbose=0,callbacks=[es])
    return min(h.history["val_loss"])

# run per country
sm = []
fs = ["imports_goods_services_gdp_pct","exports_goods_services_gdp_pct","trade_gdp_pct","inflation_consumer_prices_pct","gdp_growth_pct","trade_balance_gdp_pct","gdp_growth_stability","inflation_stability","HHI_export","Diversification_export","HHI_import","Diversification_import","Trade_Diversification_Index","Overall_Trade_Dependency"]
tg = "gdp_current_usd"

for _,r in tpd.iterrows():
    c = r["c"]
    p = r["p"]
    shr = r["shr"]
    print(f"\n==============================")
    print(f"Country: {c} | Top partner: {p} ({shr:.2f})")
    print("==============================")
    d = i[i["Country Name"]==c].copy()
    d = d.dropna(subset=fs+[tg])
    if d.empty or d["Year"].nunique()<10:
        print(f"Not enough data for {c}. Skipping.")
        continue
    sx = MinMaxScaler()
    sy = MinMaxScaler()
    X = sx.fit_transform(d[fs])
    y = sy.fit_transform(d[[tg]])
    Xs,ys = ms(X,y,3)
    if len(Xs)<10:
        print(f"Too short sequence for {c}. Skipping.")
        continue
    yrs = d["Year"].values[3:]
    trc = np.where(yrs<=2022)[0][-1]
    vc = np.where(yrs<=2024)[0][-1]
    Xtr,Ytr = Xs[:trc+1],ys[:trc+1]
    Xv,Yv = Xs[trc+1:vc+1],ys[trc+1:vc+1]
    Xt,Yt = Xs[vc+1:],ys[vc+1:]
    st = optuna.create_study(direction="minimize")
    st.optimize(lambda tr:o(tr,Xtr,Ytr,Xv,Yv,(Xtr.shape[1],Xtr.shape[2])),n_trials=20,timeout=300)
    bp = st.best_params
    print("Best params:",bp)
    bm = keras.Sequential()
    for j in range(bp["l"]):
        rs = (j<bp["l"]-1)
        if j==0:
            bm.add(layers.LSTM(bp["u"],activation="tanh",return_sequences=rs,input_shape=(Xtr.shape[1],Xtr.shape[2])))
        else:
            bm.add(layers.LSTM(bp["u"],activation="tanh",return_sequences=rs))
        bm.add(layers.Dropout(bp["d"]))
    bm.add(layers.Dense(1))
    bm.compile(optimizer=keras.optimizers.Adam(learning_rate=bp["lr"]),loss="mse")
    es = keras.callbacks.EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)
    bm.fit(np.concatenate([Xtr,Xv]),np.concatenate([Ytr,Yv]),epochs=200,batch_size=bp["bs"],verbose=0,callbacks=[es])
    ls = Xs[-1:]
    bl = bm.predict(ls,verbose=0)[0][0]
    blv = sy.inverse_transform([[bl]])[0,0]
    sh = ls.copy()
    idx = fs.index("Overall_Trade_Dependency")
    sh[0,-1,idx] *= (1-0.40*shr)
    shv = bm.predict(sh,verbose=0)[0][0]
    shv = sy.inverse_transform([[shv]])[0,0]
    loss = blv-shv
    lp = loss/blv*100
    sm.append({"c":c,"tp":p,"bl":blv,"sh":shv,"loss":loss,"lp":lp})

if sm:
    s = pd.DataFrame(sm).sort_values("lp",ascending=False)
    pd.set_option("display.float_format",lambda v:f"{v:,.2f}")
    print("\n===== GDP Impact Summary (2026, 40% drop in top partner's imports, Tuned LSTM) =====")
    print(s)
else:
    print("\n No valid countries had enough data for tuned LSTM modeling.")


[I 2025-08-19 12:50:34,709] A new study created in memory with name: no-name-d8de42bd-1209-49e7-a4b8-d813799df77c


Using features (8): ['imports_goods_services_gdp_pct', 'exports_goods_services_gdp_pct', 'trade_gdp_pct', 'inflation_consumer_prices_pct', 'gdp_growth_pct', 'trade_balance_gdp_pct', 'gdp_growth_stability', 'inflation_stability']
20 countries available for training.
Starting Optuna tuning (this may take a few minutes)...


[I 2025-08-19 12:50:44,171] Trial 0 finished with value: 0.001070842845365405 and parameters: {'seq_len': 6, 'units': 111, 'n_layers': 1, 'dropout': 0.23037268093189525, 'lr': 0.008704452829501101, 'batch_size': 64}. Best is trial 0 with value: 0.001070842845365405.
[I 2025-08-19 12:51:11,919] Trial 1 finished with value: 0.0022032465785741806 and parameters: {'seq_len': 5, 'units': 87, 'n_layers': 2, 'dropout': 0.3886832540440106, 'lr': 0.0005266794604241971, 'batch_size': 32}. Best is trial 0 with value: 0.001070842845365405.
[I 2025-08-19 12:51:19,634] Trial 2 finished with value: 0.006389304995536804 and parameters: {'seq_len': 7, 'units': 93, 'n_layers': 1, 'dropout': 0.04236702868169387, 'lr': 0.0008461530081708932, 'batch_size': 32}. Best is trial 0 with value: 0.001070842845365405.
[I 2025-08-19 12:51:26,139] Trial 3 finished with value: 0.00955226644873619 and parameters: {'seq_len': 4, 'units': 60, 'n_layers': 1, 'dropout': 0.19972785876753668, 'lr': 0.0026523743541563183, 'b

Best trial params:
{'seq_len': 7, 'units': 44, 'n_layers': 1, 'dropout': 0.24159158618946538, 'lr': 0.005061541402477744, 'batch_size': 16}


Round 1: max incremental GDP% = 2.078412
Round 2: max incremental GDP% = 2.808705
Round 3: max incremental GDP% = 4.013193
Round 4: max incremental GDP% = 5.933151
Round 5: max incremental GDP% = 8.945497
Round 6: max incremental GDP% = 13.969789
Round 7: max incremental GDP% = 22.047909
Round 8: max incremental GDP% = 34.621623
Round 9: max incremental GDP% = 54.165284
Round 10: max incremental GDP% = 84.512063
Round 11: max incremental GDP% = 131.599276
Round 12: max incremental GDP% = 204.622538
Round 13: max incremental GDP% = 317.822966
Round 14: max incremental GDP% = 493.254607
Round 15: max incremental GDP% = 765.069692
Round 16: max incremental GDP% = 1186.154168
Round 17: max incremental GDP% = 1838.402462
Round 18: max incremental GDP% = 2848.627423
Round 19: max incremental GDP% = 4413.195979
Round 20: max incremental GDP% = 6836.176045

=== Top 5 countries by GDP % loss after China 25% export shock and propagation ===
    country  gdp_loss_pct     gdp_loss_abs_usd  export_